In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style

# from mlxtend.frequent_patterns import apriori,association_rules
from collections import Counter
from IPython.display import Image

In [2]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=NameError)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [3]:
products = pd.read_excel("Data/Rec_sys_data.xlsx", sheet_name="product")
products = products.dropna()

In [6]:
products.shape

(3706, 6)

In [7]:
products.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


In [4]:
products_2 = products.drop_duplicates(subset=["Unit Price", "Brand", "Description"], keep="first")

In [25]:
products_2.shape

(1506, 6)

In [28]:
list(products_2["StockCode"])

[22629,
 21238,
 22181,
 84879,
 84836,
 22716,
 'POST',
 23357,
 'M',
 23283,
 22098,
 21621,
 23355,
 21155,
 '72800E',
 23110,
 22952,
 21246,
 20972,
 22037,
 22301,
 85178,
 '85180B',
 22139,
 22402,
 23343,
 23661,
 21498,
 84991,
 21445,
 84978,
 21264,
 20728,
 22970,
 20719,
 82486,
 21932,
 22734,
 84378,
 23345,
 '85131C',
 23084,
 21259,
 21864,
 82600,
 23193,
 21217,
 84012,
 22652,
 23424,
 21232,
 23236,
 22776,
 23217,
 '85099F',
 21668,
 23245,
 22625,
 22866,
 21086,
 21684,
 23321,
 22906,
 22326,
 20676,
 21992,
 '85099B',
 21928,
 21802,
 '79063D',
 22470,
 71477,
 22165,
 22677,
 22895,
 22857,
 23206,
 79321,
 22609,
 '47559b',
 22526,
 23007,
 22624,
 22966,
 21080,
 22277,
 22085,
 23298,
 22437,
 22697,
 23351,
 22620,
 23120,
 21871,
 21428,
 21791,
 21915,
 21936,
 23235,
 21524,
 21658,
 '16258A',
 22173,
 48194,
 22502,
 22086,
 21128,
 22554,
 22327,
 21874,
 20677,
 23012,
 23297,
 23083,
 23192,
 23581,
 22042,
 22745,
 '84970S',
 22766,
 21068,
 20726

In [29]:
categories = [{"name": p.split("|")[0], "storeId": 1, "billboardId": 1} for p in products_2.Category.values]

In [30]:
def increment_counter():
    if not hasattr(increment_counter, "counter"):
        increment_counter.counter = 55  # Initialize it for the first time
    increment_counter.counter += 1
    return increment_counter.counter
increment_counter()

56

In [31]:
unique_cat = set(item["name"] for item in categories)
categories = [{"id": increment_counter(), "name": c, "storeId": 1, "billboardId": 1, "createdAt": "DATE", "updatedAt": "DATE"} for c in unique_cat]
len(categories), len(unique_cat)

(29, 29)

In [32]:
categories[0]

{'id': 57,
 'name': 'Baby',
 'storeId': 1,
 'billboardId': 1,
 'createdAt': 'DATE',
 'updatedAt': 'DATE'}

In [33]:
with open('categories.txt', 'w') as file:
    for dictionary in categories:
        # Write the dictionary as a string
        file.write(str(dictionary) + ',\n')

In [34]:
products = products_2.rename(columns={"Product Name": "name",  "Description": "description",
         "StockCode": "stockCode",
        "Unit Price": "price",
        "Brand": "brand",
        "Category": "category"})

In [35]:
def increment_counter():
    if not hasattr(increment_counter, "counter"):
        increment_counter.counter = 55  # Initialize it for the first time
    increment_counter.counter += 1
    return increment_counter.counter
increment_counter()

56

In [36]:
def get_id(id):
    try:
        return int(id)
    except:
        return increment_counter()

In [37]:
new_products = [{
        "name": row["name"],
        "description": row["description"],
        "stockCode": row["stockCode"],
        "price": row["price"],
        "brand": row["brand"],
        "category": row["category"].split("|")[0]
    } for index, row in products.iterrows()]

In [38]:
def find_item(list_of_dicts, key, value):
    for item in list_of_dicts:
        if item.get(key) == value:
            return item["id"]
    return None

In [39]:
temp = [{"name": row["name"],
        "description": row["description"],
        "stockCode": str(row["stockCode"]),
        "id": get_id(row["stockCode"]),
        "price": row["price"],
        "brand": row["brand"],
        "isFeatured": True,
        "isArchived": True,
        "storeId": 1,
        "createdAt": "DATE-TIME",
        "updatedAt": "DATE-TIME",
        "categoryId": find_item(categories, "name", row["category"])
        } for row in new_products]

In [40]:
len(new_products)

1506

In [41]:
with open('products.txt', 'w') as file:
    for dictionary in temp:
        # Write the dictionary as a string
        file.write(str(dictionary) + ',\n')

In [9]:
import ast

def read_and_parse_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read().strip()

    # Remove the trailing comma and newline if present
    if content.endswith(','):
        content = content[:-1]

    # Split the content by '},\n' to separate each dictionary
    raw_items = content.split('},\n')

    items = []
    for raw_item in raw_items:
        # Add back the removed '}' (except for the last item)
        if not raw_item.endswith('}'):
            raw_item = raw_item + '}'
        # Use ast.literal_eval to safely evaluate the string to a dictionary
        item = ast.literal_eval(raw_item.strip())
        items.append(item)
    
    return items

In [10]:
items = read_and_parse_file("products.txt")

In [14]:
items[0]

{'name': 'Ganma Superheroes Ordinary Life Case For Samsung Galaxy Note 5 Hard Case Cover',
 'description': 'New unique design, great gift.High quality plastic material.Clips over the back of the device to protect the back &amp; sides from bumps and scratches.Printed using a process called sublimation, high quality image which will last for years!| New unique design, great gift. High quality plastic material. Clips over the back of the device to protect the back &amp; sides from bumps and scratches. Printed using a process called sublimation, high quality image which will last for years! ',
 'stockCode': '22629',
 'id': 22629,
 'price': 13.99,
 'brand': 'Ganma',
 'isFeatured': True,
 'isArchived': True,
 'storeId': 1,
 'createdAt': 'DATE-TIME',
 'updatedAt': 'DATE-TIME',
 'categoryId': 70}

In [15]:
ids = [item["id"] for item in items]

In [20]:
with open ("productIds", "w") as file:
    for id in ids:
        file.write(str(id) + ",")